In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_csv(r"C:\Users\mixas\Documents\Housing.csv")
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
df_encoded = pd.get_dummies(df, columns=["mainroad", "guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea", "furnishingstatus"])
df_encoded.head()

,price,area,bedrooms,bathrooms,stories,parking,mainroad_no,mainroad_yes,guestroom_no,guestroom_yes,...,basement_yes,hotwaterheating_no,hotwaterheating_yes,airconditioning_no,airconditioning_yes,prefarea_no,prefarea_yes,furnishingstatus_furnished,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
0,13300000,7420,4,2,3,2,False,True,True,False,...,False,True,False,False,True,False,True,True,False,False
1,12250000,8960,4,4,4,3,False,True,True,False,...,False,True,False,False,True,True,False,True,False,False
2,12250000,9960,3,2,2,2,False,True,True,False,...,True,True,False,True,False,False,True,False,True,False
3,12215000,7500,4,2,2,3,False,True,True,False,...,True,True,False,False,True,False,True,True,False,False
4,11410000,7420,4,1,2,2,False,True,False,True,...,True,True,False,False,True,True,False,True,False,False


In [4]:
np.sum(pd.isna(df_encoded) == True) # checking if any of the columns has NaN datapoints

C:\Users\mixas\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


price                              0
area                               0
bedrooms                           0
bathrooms                          0
stories                            0
parking                            0
mainroad_no                        0
mainroad_yes                       0
guestroom_no                       0
guestroom_yes                      0
basement_no                        0
basement_yes                       0
hotwaterheating_no                 0
hotwaterheating_yes                0
airconditioning_no                 0
airconditioning_yes                0
prefarea_no                        0
prefarea_yes                       0
furnishingstatus_furnished         0
furnishingstatus_semi-furnished    0
furnishingstatus_unfurnished       0
dtype: int64

In [5]:
df_encoded = df_encoded.replace({True: 1, False: 0}).astype(float)
X = df_encoded.to_numpy()
X.dtype

C:\Users\mixas\AppData\Local\Temp\ipykernel_5188\3533865153.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_encoded = df_encoded.replace({True: 1, False: 0}).astype(float)


dtype('float64')

In [6]:
X = df_encoded.drop(["price"], axis = 1)
y = df_encoded["price"]
X.shape

(545, 20)

In [7]:
# Normalizing continuos data for better predictions
mean_area = X["area"].mean()
mean_price = y.mean()
std_area = X["area"].std()
std_price = y.std()
X["area"] = (X["area"] - mean_area) / std_area
y = (y - mean_price) / std_price

mean_bedrom = X["bedrooms"].mean()
std_bed = X["bedrooms"].std()
X["bedrooms"] = (X["bedrooms"] - mean_bedrom) / std_bed

mean_bath = X["bathrooms"].mean()
std_bath = X["bathrooms"].std()
X["bathrooms"] = (X["bathrooms"] - mean_bath) / std_bath

mean_stories = X["stories"].mean()
std_stories = X["stories"].std()
X["stories"] = (X["stories"] - mean_stories) / std_stories

mean_parking= X["parking"].mean()
std_parking = X["parking"].std()
X["parking"] = (X["parking"] - mean_parking) / std_parking


In [8]:
from sklearn.model_selection import train_test_split
# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# the model only takes numpy arrays so we need to conver dataframe to array
X_train = np.asarray(X_train.T)        # (features, m)
y_train = np.asarray(y_train).reshape(1, -1)  # (1, m) for BCE/MSE

In [10]:
from nn_constructor import NeuralNetwork, Layer, LinearAct, ReluAct, MeanRegressionError

# creating layers for the model
layer1 = Layer(in_features=20, out_features=12, activation=ReluAct())
layer2 = Layer(in_features=12, out_features=8, activation=ReluAct())
layer3 = Layer(in_features=8, out_features=4, activation=ReluAct() ) 
layer4 = Layer(in_features=4, out_features=1, activation=LinearAct() ) 

# Building network
layers = [layer1, layer2, layer3, layer4]

# Initialize loss
loss = MeanRegressionError(m=X_train.shape[1], y=y_train, lamda=0.01)

# Build NeuralNetwork object
nn = NeuralNetwork(alpha=0.001, data=X_train, loss=loss, layers=layers)

nn.fit()

epoch 1 - cost 32877.5253
epoch 2 - cost 23536.089
epoch 3 - cost 17361.711
epoch 4 - cost 13075.5397
epoch 5 - cost 9995.5067
epoch 6 - cost 7725.6477
epoch 7 - cost 6020.9923
epoch 8 - cost 4722.2994
epoch 9 - cost 3721.8975
epoch 10 - cost 2944.6303
epoch 11 - cost 2336.6605
epoch 12 - cost 1858.5967
epoch 13 - cost 1481.1124
epoch 14 - cost 1182.0622
epoch 15 - cost 944.5285
epoch 16 - cost 755.4643
epoch 17 - cost 604.7302
epoch 18 - cost 484.3969
epoch 19 - cost 388.2321
epoch 20 - cost 311.3175
epoch 21 - cost 249.7584
epoch 22 - cost 200.4631
epoch 23 - cost 160.9716
epoch 24 - cost 129.3235
epoch 25 - cost 103.9542
epoch 26 - cost 83.6136
epoch 27 - cost 67.3022
epoch 28 - cost 54.2201
epoch 29 - cost 43.7269
epoch 30 - cost 35.3094
epoch 31 - cost 28.5567
epoch 32 - cost 23.1391
epoch 33 - cost 18.7926
epoch 34 - cost 15.3052
epoch 35 - cost 12.5071
epoch 36 - cost 10.262
epoch 37 - cost 8.4605
epoch 38 - cost 7.015
epoch 39 - cost 5.8552
epoch 40 - cost 4.9245
epoch 41 - cos

In [11]:
X_test =np.asarray( X_test.T) 
prediction = nn.predict(X_test) # making predictions on X_test

In [12]:
y_test = np.asarray(y_test).reshape(1, -1)
nn.meansquarederror(prediction, y_test) # checking the accuracy of predictions on test data

2.6185378786985387